### Importaciones y descarga

In [ ]:
!pip install spacy
!pyhton -m spacydownload es_core_news_sm

In [1]:
#Librerias (import)
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('Corpus-Agro.xlsx')
df.head()

### Analisis Morfologico 

In [ ]:
import es_core_news_sm
from string import punctuation

In [ ]:
#Leer el archivo de texto en español (ajuste ingles)
def LeerTexto(NombreArchivo):
    f = open(NombreArchivo, 'r', encoding="utf8")
    texto = f.read().split('.')
    f.close()
    return(texto)

In [ ]:
#Lematzacion, generamos los lemas
def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))  

In [ ]:
#Cargamos los datos
NOMBRE_ARCHIVO='sample.txt'
nlp       = es_core_news_sm.load()

#Leemos el texto 
oraciones = LeerTexto(NOMBRE_ARCHIVO)
print(oraciones)

In [ ]:
#Mostramos lista de con las oraciones lematizadas
texto_lematizado  = [Lematizar(oracion) for oracion in oraciones]
print(texto_lematizado)

### Analisis Lexico

In [ ]:
import es_core_news_sm
import re

In [ ]:
#Definimos la funcion de etiquetado de texto
def Etiquetar(texto):
   doc = nlp(texto)
   Etiquetado = ''.join(t.text+"/"+t.pos_+" " for t in doc)
   return(Etiquetado.rstrip())

In [ ]:
#inicia variables y metodos
NombreArchivo='sample.txt'  
nlp = es_core_news_sm.load()

#Etiquetamos
texto = open(NombreArchivo, 'r',encoding="utf8").read()
texto_etiquetado = Etiquetar(texto)

In [ ]:
#Extraemos tiodas las palabras de etiquetas

# La expresión para cualquier palabra es: "(\w+)"
Calces  = re.findall("(\w+)/NOUN",texto_etiquetado)
print(Calces)